In [1]:
import os
import tempfile
from pprint import pprint

from cognite.powerops import PowerOpsClient
from cognite.powerops.client.shop.data_classes import Case

**Note** this is an older demo and might not be relevant any more

In [2]:
# You can control which setting files are loaded through the environmental variable below. 
# In this case, the setting files are located two levels above.
os.environ["SETTINGS_FILES"] = "../../settings.toml;../../.secrets.toml"

powerops = PowerOpsClient.from_settings()

In [4]:
# we'll be downloading files, need a dir:
# (using tempfile here, but anything works)
tmp_dir = tempfile.mkdtemp(prefix="power-ops-sdk-usage")

# get a file:
file_meta = powerops.cdf.files.retrieve(external_id="SHOP_Fornebu_model")
powerops.cdf.files.download(directory=tmp_dir, external_id="SHOP_Fornebu_model")
case_file = os.path.join(tmp_dir, file_meta.name)

# load the file:
case = Case.from_yaml_file(case_file)

# get a value:
print(case.data["model"]["creek_intake"]["Golebiowski_intake"]["net_head"])

# get a list:
print(case.data["model"]["generator"]["Holen_G1"]["gen_eff_curve"]["y"])

# modify a list: a gen_eff_curve, add +10 to Y:
case.data["model"]["generator"]["Holen_G1"]["gen_eff_curve.y"] = [
    val + 10 for val in case.data["model"]["generator"]["Holen_G1"]["gen_eff_curve"]["y"]
]
# TODO ^ Cast lists to pd.Series automatically? Then need to handle setter, too.

# export yaml:
edited_case_file = os.path.join(tmp_dir, "edited_case.yaml")
case.save_yaml(edited_case_file)

# (just for show) load that exported yaml and check the edited curve:
case2 = Case.from_yaml_file(edited_case_file)
pprint(case2.data["model"]["generator"]["Holen_G1"]["gen_eff_curve"])

753
[97, 97.5, 97.9, 98.2, 98.3, 98.43, 98.6, 98.64, 98.65, 98.63]
{'ref': 0,
 'x': [15, 16.9, 21.8, 25.8, 28.2, 32.5, 38.5, 41.5, 43, 44],
 'y': [97, 97.5, 97.9, 98.2, 98.3, 98.43, 98.6, 98.64, 98.65, 98.63]}
